In [1]:
from aiy.vision.inference import CameraInference
from aiy.vision.models import object_detection
from aiy.vision.streaming.server import StreamingServer
from aiy.vision.streaming import svg
from aiy.leds import Leds, Color
from gpiozero import Servo
from aiy.pins import PIN_A

from picamera import PiCamera
from IPython.display import Image, display, clear_output

import contextlib
import time

### Animal Species Detector
This should be fun :)

In [2]:
servo = Servo(PIN_A)

/usr/lib/python3/dist-packages/gpiozero/output_devices.py:1533: PWMSoftwareFallback: To reduce servo jitter, use the pigpio pin factory.See https://gpiozero.readthedocs.io/en/stable/api_output.html#servo for more info
  'To reduce servo jitter, use the pigpio pin factory.'


I added some code here that is called to create the overlay - basically the box around the face and the score above. You can customize it and/or add information you want to overlay on the camera feed. **This is not needed for the servo.**

In [17]:
def svg_overlay(objects, frame_size):

    #JOY_COLOR = (255, 70, 0)
    #SAD_COLOR = (0, 0, 64)

    labels = ["None", "Person", "Cat", "Dog"]
    width, height = frame_size
    doc = svg.Svg(width=width, height=height)

    for obj in objects:
        x, y, w, h = obj.bounding_box
        doc.add(svg.Rect(x=int(x), y=int(y), width=int(w), height=int(h), rx=10, ry=10,
                         fill_opacity=0.3,
                         style='stroke:white;stroke-width:4px'))

        doc.add(svg.Text(labels[obj.kind], x=x, y=y-10, fill='white', font_size=30))

    return str(doc)

#### Main loop

Here is our main loop based on the code we used last time. Look at the comments on what was changed. Basically we removed the part saving the picture and adjust the servo value to the joy score each frame. We also added the streaming back in, to while this cell runs, you can connect to http://orcspi-vis.local:4664 and see the stream.

In [39]:
with contextlib.ExitStack() as stack:
    leds   = stack.enter_context(Leds())
    camera = stack.enter_context(PiCamera(sensor_mode=4, resolution=(820, 616)))

    # This starts and runs the streaming of the camera
    server = stack.enter_context(StreamingServer(camera))  

    print ("Loading model - hold on ..")
    
    # Move the servo to low (-1)
    servo.min()
    
    # Do inference on VisionBonnet
    with CameraInference(object_detection.model()) as inference:
        try:   
            for result in inference.run():
                leds.update(Leds.rgb_on(Color.BLACK))
                objects = object_detection.get_objects(result)
                
                # This sends the overlay (boxes) to add to the camera stream
                server.send_overlay(svg_overlay(objects, (result.width, result.height)))

                if len(objects) >= 1:
                    clear_output(wait=True)                 
                    leds.update(Leds.rgb_on(Color.PURPLE))
                    print("Objects: ", objects[0]) 
                    
                    # This sets the servo to the joy score value
                    # To use the full range (-1 to 1) we multiply the score by 2 and subtract 1
                    #servo.value = 2*faces[0].joy_score  - 1
                                           
        except KeyboardInterrupt:
            print("Interrupted ..")
            
    leds.update(Leds.rgb_off())
    
    # Servo back to the middle upon ending
    servo.mid()
    
    print("Done")

Objects:  kind=CAT(2), score=0.645232, bbox=(248, 313, 591, 914)
Interrupted ..
Done
